In [1]:
!pip cache purge
!pip install torch

Files removed: 0


In [148]:
import string 
import numpy as np
import torch
import random
from tqdm import tqdm
from util import load_train_data, load_test_data


# Load data
positive_file = "hotelPosT-train.txt"
negative_file = "hotelNegT-train.txt"
all_texts, all_labels = load_train_data(positive_file, negative_file)
test_file = "HW2-testset.txt"

test_texts, test_labels = load_test_data(test_file)

# Count positive and negative samples
num_positive = sum(all_labels)
num_negative = len(all_labels) - num_positive

print(f"Total reviews: {len(all_texts)}")
print(f"Positive reviews: {num_positive}")
print(f"Negative reviews: {num_negative}")

# Count positive and negative samples
num_test = sum(test_labels)

print(f"Total Test reviews: {len(test_texts)}")
print(f"Tiotal reviews: {num_test}")


Total reviews: 189
Positive reviews: 95
Negative reviews: 94
Total Test reviews: 50
Tiotal reviews: 25


In [149]:
from sklearn.model_selection import train_test_split

train_texts, dev_texts, train_labels, dev_labels = train_test_split(
    all_texts, all_labels, test_size=0.2, random_state=42
)


print(f"Training set size: {len(train_texts)}")
print(f"Development set size: {len(dev_texts)}")


Training set size: 151
Development set size: 38


In [150]:
def precision(predicted_labels, true_labels):
    tp = sum(p == 1 and t == 1 for p, t in zip(predicted_labels, true_labels))
    fp = sum(p == 1 and t == 0 for p, t in zip(predicted_labels, true_labels))
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def recall(predicted_labels, true_labels):
    tp = sum(p == 1 and t == 1 for p, t in zip(predicted_labels, true_labels))
    fn = sum(p == 0 and t == 1 for p, t in zip(predicted_labels, true_labels))
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def f1(predicted_labels, true_labels):
    p = precision(predicted_labels, true_labels)
    r = recall(predicted_labels, true_labels)
    return 2 * p * r / (p + r) if (p + r) > 0 else 0

# Example test case
true_labels = [1, 1, 1, 1, 0, 1, 0, 0, 1, 0]
predicted_labels = [1, 0, 1, 0, 0, 1, 1, 0, 1, 0]

print("Precision:", precision(predicted_labels, true_labels))
print("Recall:", recall(predicted_labels, true_labels))
print("F1 Score:", f1(predicted_labels, true_labels))


Precision: 0.8
Recall: 0.6666666666666666
F1 Score: 0.7272727272727272


In [151]:
def featurize_text(text):
    # Tokenize 
    words = text.lower().split()
    positive_count = sum(1 for word in words if word in positive_lexicon)
    negative_count = sum(1 for word in words if word in negative_lexicon)
    contains_no = 1.0 if "no" in words else 0.0
    pronouns = {"i", "me", "them", "they", "my", "mine", "we", "us", "our", "ours", "you", "your", "yours"}
    pronoun_count = sum(1 for word in words if word in pronouns)
    contains_exclamation = 1.0 if "!" in text else 0.0
    log_word_count = math.log(len(words)) if len(words) > 0 else 0.0
    feature_vector = [positive_count, negative_count, contains_no, pronoun_count, contains_exclamation, log_word_count]
    return feature_vector

In [152]:
def load_lexicon(file_path):
    with open(file_path, 'r') as f:
        return set(line.strip() for line in f.readlines())

positive_lexicon = load_lexicon('positive-words.txt')
negative_lexicon = load_lexicon('negative-words.txt')

In [153]:
train_vectors = [featurize_text(review) for review in train_texts]
all_dev_feature_vectors = [featurize_text(review) for review in dev_texts]
all_test_feature_vectors = [featurize_text(review) for review in test_texts]

In [154]:
def normalize(feature_vectors):
    feature_vectors = np.array(feature_vectors)
    
    min_vals = feature_vectors.min(axis=0)
    max_vals = feature_vectors.max(axis=0)
    normalized = (feature_vectors - min_vals) / (max_vals - min_vals)
    
    return normalized.tolist()


In [155]:
train_vectors = normalize(train_vectors)
all_dev_feature_vectors = normalize(all_dev_feature_vectors)
all_test_feature_vectors = normalize(all_test_feature_vectors)
print("\nOriginal Feature Vectors for Train Texts(for the first 5 reviews):")
for i, vec in enumerate(train_vectors[:5]):  # Display first 5 for brevity
    print(f"Review {i+1}: {vec}")

print("\nNormalized Feature Vectors for Train Texts(for the first 5 reviews):")
for i, vec in enumerate(train_vectors[:5]):  # Display first 5 for brevity
    print(f"Review {i+1}: {vec}")
    
print("\nOriginal Feature Vectors for Dev Texts(for the first 5 reviews):")
for i, vec in enumerate(all_dev_feature_vectors[:5]):  # Display first 5 for brevity
    print(f"Review {i+1}: {vec}")

print("\nNormalized Feature Vectors for Dev Texts(for the first 5 reviews):")
for i, vec in enumerate(all_dev_feature_vectors[:5]):  # Display first 5 for brevity
    print(f"Review {i+1}: {vec}")
    
print("\nOriginal Feature Vectors for Test Texts(for the first 5 reviews):")
for i, vec in enumerate(all_test_feature_vectors[:5]):  # Display first 5 for brevity
    print(f"Review {i+1}: {vec}")

print("\nNormalized Feature Vectors for Test Texts(for the first 5 reviews):")
for i, vec in enumerate(all_test_feature_vectors[:5]):  # Display first 5 for brevity
    print(f"Review {i+1}: {vec}")


Original Feature Vectors for Train Texts(for the first 5 reviews):
Review 1: [0.24, 0.0, 0.05263157894736842, 0.7533293705497731, 0.041666666666666664, 0.07142857142857142]
Review 2: [0.12, 0.5, 0.2425629290617849, 0.19225549836219014, 0.375, 0.26785714285714285]
Review 3: [0.04, 0.25, 0.1853546910755149, 0.3932073984680937, 0.041666666666666664, 0.19642857142857142]
Review 4: [0.56, 0.0, 0.32036613272311215, 0.7985827929674446, 0.08333333333333333, 0.4107142857142857]
Review 5: [0.24, 0.25, 0.10297482837528604, 0.06674174948248364, 0.125, 0.14285714285714285]

Normalized Feature Vectors for Train Texts(for the first 5 reviews):
Review 1: [0.24, 0.0, 0.05263157894736842, 0.7533293705497731, 0.041666666666666664, 0.07142857142857142]
Review 2: [0.12, 0.5, 0.2425629290617849, 0.19225549836219014, 0.375, 0.26785714285714285]
Review 3: [0.04, 0.25, 0.1853546910755149, 0.3932073984680937, 0.041666666666666664, 0.19642857142857142]
Review 4: [0.56, 0.0, 0.32036613272311215, 0.79858279296744

In [156]:
'''
CSCI 5832 Assignment 2
Spring 2025
The following sample code was taken from a tutorial by PyTorch and modified for our assignment.
Source: https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html
'''
class SentimentClassifier(torch.nn.Module):

    def __init__(self, input_dim: int = 6, output_size: int = 1):
        super(SentimentClassifier, self).__init__()

        # Define the parameters that we will need.
        # Torch defines nn.Linear(), which gives the linear function z = Xw + b.
        self.linear = torch.nn.Linear(input_dim, output_size)

    def forward(self, feature_vec):
        # Pass the input through the linear layer,
        # then pass that through sigmoid to get a probability.
        z = self.linear(feature_vec)
        return torch.sigmoid(z)
    
    def logprob2label(self, prob):
        """Convert probability to a binary label (0 or 1)"""
        return (prob >= 0.5).int()
    

model = SentimentClassifier()

# the model knows its parameters.  The first output below is X, the second is b.
# Whenever you assign a component to a class variable in the __init__ function
# of a module, which was done with the line
# self.linear = nn.Linear(...)
# Then through some Python magic from the PyTorch devs, your module
# (in this case, SentimentClassifier) will store knowledge of the nn.Linear's parameters
for param in model.parameters():
    print(param)

# To run the model, pass in a feature vector
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    sample_feature_vector = torch.tensor([[3.0, 2.0, 1.0, 3.0, 0.0, 4.18965482711792],[3.0, 2.0, 1.0, 3.0, 0.0, 4.18965482711792]])
    log_prob = model(sample_feature_vector)
    print('Log probability from the untrained model:', log_prob)
    print('Label based on the log probability:', model.logprob2label(log_prob))


# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# Sample training loop below. Because it uses functions that you are asked to write for the assignment,     #
# it will not run as is, and is not guaranteed to work with your existing code. You may need to modify it.  #
#                                                                                                           #
# No need to use this code if you have a better way,                                                        #
# or if you can't figure out how to make it run with your existing code.                                    #
# It is only provided here to give you an idea of how we expect you to train the model.                     #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #



loss_function = torch.nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
num_epochs = 100
batch_size = 16


# Featurize and normalize
# train_vectors_test = [featurize_text(text) for text in train_texts]  
# train_vectors_test = normalize(train_vectors_test)  

train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1)  

for epoch in range(num_epochs):
    samples = list(zip(train_vectors, train_labels_tensor))
    random.shuffle(samples)
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]

    epoch_i_train_losses = []

    for batch in tqdm(batches):
        feature_vectors, labels = zip(*batch)
        
        # Convert to tensors
        feature_vectors = torch.tensor(feature_vectors, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Ensure labels are 2D
        
        # Step 1: Zero gradients
        model.zero_grad()

        # Step 2: Forward pass
        log_probs = model(feature_vectors)

        # Step 3: Compute loss
        loss = loss_function(log_probs, labels)

        # Step 4: Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Store loss
        epoch_i_train_losses.append(loss.item())
    
    # Print the average loss for this epoch
    print('Epoch:', epoch)
    print('Avg train loss:', sum(epoch_i_train_losses) / len(epoch_i_train_losses))

Parameter containing:
tensor([[-0.2572, -0.2950,  0.3682, -0.0405, -0.2338,  0.0362]],
       requires_grad=True)
Parameter containing:
tensor([-0.1510], requires_grad=True)
Log probability from the untrained model: tensor([[0.2471],
        [0.2471]])
Label based on the log probability: tensor([[0],
        [0]], dtype=torch.int32)


100%|██████████| 10/10 [00:00<00:00, 1497.06it/s]


Epoch: 0
Avg train loss: 0.6845852732658386


100%|██████████| 10/10 [00:00<00:00, 1369.30it/s]


Epoch: 1
Avg train loss: 0.6738928496837616


...

Epoch: 97
Avg train loss: 0.43692323863506316


100%|██████████| 10/10 [00:00<00:00, 1860.66it/s]


Epoch: 98
Avg train loss: 0.44408890306949617


100%|██████████| 10/10 [00:00<00:00, 1562.30it/s]


Epoch: 99
Avg train loss: 0.4335035473108292


In [157]:
num_epochs = 100
batch_size = 16
dev_labels_tensor = torch.tensor(dev_labels, dtype=torch.float32).unsqueeze(1) 
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
for epoch in range(num_epochs):
    samples = list(zip(all_dev_feature_vectors, dev_labels_tensor))
    random.shuffle(samples)
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]

    epoch_i_train_losses = []

    for batch in tqdm(batches):
        feature_vectors, labels = zip(*batch)
        
        # Convert to tensors
        feature_vectors = torch.tensor(feature_vectors, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Ensure labels are 2D
        
        # Step 1: Zero gradients
        model.zero_grad()

        # Step 2: Forward pass
        log_probs = model(feature_vectors)

        # Step 3: Compute loss
        loss = loss_function(log_probs, labels)

        # Step 4: Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Store loss
        epoch_i_train_losses.append(loss.item())
    
    # Print the average loss for this epoch
    print('Epoch:', epoch)
    print('Avg train loss:', sum(epoch_i_train_losses) / len(epoch_i_train_losses))


100%|██████████| 3/3 [00:00<00:00, 898.07it/s]


Epoch: 0
Avg train loss: 0.44176697731018066


100%|██████████| 3/3 [00:00<00:00, 886.12it/s]


Epoch: 1
Avg train loss: 0.4638017416000366


100%|██████████| 3/3 [00:00<00:00, 1327.17it/s]


...

Epoch: 97
Avg train loss: 0.470263530810674


100%|██████████| 3/3 [00:00<00:00, 1316.62it/s]


Epoch: 98
Avg train loss: 0.45249488949775696


100%|██████████| 3/3 [00:00<00:00, 1191.56it/s]

Epoch: 99
Avg train loss: 0.4453155001004537


In [158]:
from sklearn.metrics import precision_score, recall_score, f1_score

dev_feature_tensor = torch.tensor(all_dev_feature_vectors, dtype=torch.float32)
true_dev_labels_tensor = torch.tensor(dev_labels, dtype=torch.int)

# Get predicted probabilities from the model
with torch.no_grad():  # No gradient calculation needed for evaluation
    dev_probs = model(dev_feature_tensor)

# Convert probabilities to binary labels
predicted_dev_labels = model.logprob2label(dev_probs)

# Convert tensors to lists for metric calculation
predicted_dev_labels = predicted_dev_labels.squeeze().tolist()
true_dev_labels = true_dev_labels_tensor.tolist()

# Calculate precision, recall, and F1-score
precision = precision_score(true_dev_labels, predicted_dev_labels)
recall = recall_score(true_dev_labels, predicted_dev_labels)
f1 = f1_score(true_dev_labels, predicted_dev_labels)

# Print results
print(f'Precision for Dev: {precision:.4f}')
print(f'Recall for Dev: {recall:.4f}')
print(f'F1-score for Dev: {f1:.4f}')

Precision for Dev: 0.9000
Recall for Dev: 0.9474
F1-score for Dev: 0.9231


In [159]:

num_epochs = 200
optimizer = torch.optim.SGD(model.parameters(), lr=0.05)
batch_size = 16
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32).unsqueeze(1)  

for epoch in range(num_epochs):
    samples = list(zip(all_test_feature_vectors, test_labels_tensor))
    random.shuffle(samples)
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]

    epoch_i_train_losses = []

    for batch in tqdm(batches):
        feature_vectors, labels = zip(*batch)
        
        # Convert to tensors
        feature_vectors = torch.tensor(feature_vectors, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Ensure labels are 2D
        
        # Step 1: Zero gradients
        model.zero_grad()

        # Step 2: Forward pass
        log_probs = model(feature_vectors)

        # Step 3: Compute loss
        loss = loss_function(log_probs, labels)

        # Step 4: Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Store loss
        epoch_i_train_losses.append(loss.item())
    
    # Print the average loss for this epoch
    print('Epoch:', epoch)
    print('Avg train loss:', sum(epoch_i_train_losses) / len(epoch_i_train_losses))

100%|██████████| 4/4 [00:00<00:00, 1091.49it/s]


Epoch: 0
Avg train loss: 0.4302847534418106


100%|██████████| 4/4 [00:00<00:00, 1149.36it/s]


Epoch: 1
Avg train loss: 0.40590767562389374


100%|██████████| 4/4 [00:00<00:00, 984.81it/s]


...

Epoch: 197
Avg train loss: 0.3880877047777176


100%|██████████| 4/4 [00:00<00:00, 898.47it/s]


Epoch: 198
Avg train loss: 0.34847117215394974


100%|██████████| 4/4 [00:00<00:00, 855.50it/s]


Epoch: 199
Avg train loss: 0.42023393511772156


In [160]:
test_feature_tensor = torch.tensor(all_test_feature_vectors, dtype=torch.float32)
true_test_labels_tensor = torch.tensor(test_labels, dtype=torch.int)

# Get predicted probabilities from the model
with torch.no_grad():  # No gradient calculation needed for evaluation
    test_probs = model(test_feature_tensor)

# Convert probabilities to binary labels
predicted_test_labels = model.logprob2label(test_probs)

# Convert tensors to lists for metric calculation
predicted_test_labels = predicted_test_labels.squeeze().tolist()
true_test_labels = true_test_labels_tensor.tolist()

# Calculate precision, recall, and F1-score
precision = precision_score(true_test_labels, predicted_test_labels)
recall = recall_score(true_test_labels, predicted_test_labels)
f1 = f1_score(true_test_labels, predicted_test_labels)

# Print results
print(f'Precision for Test: {precision:.4f}')
print(f'Recall for Test: {recall:.4f}')
print(f'F1-score for Test: {f1:.4f}')


Precision for Test: 0.8462
Recall for Test: 0.8800
F1-score for Test: 0.8627


In [161]:
all_test_feature_vectors_wthot_nrmlz = [featurize_text(review) for review in test_texts]

In [162]:
num_epochs = 100
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
batch_size = 16
for epoch in range(num_epochs):
    samples = list(zip(all_test_feature_vectors_wthot_nrmlz, test_labels_tensor))
    random.shuffle(samples)
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]

    epoch_i_train_losses = []

    for batch in tqdm(batches):
        feature_vectors, labels = zip(*batch)
        
        # Convert to tensors
        feature_vectors = torch.tensor(feature_vectors, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Ensure labels are 2D
        
        # Step 1: Zero gradients
        model.zero_grad()

        # Step 2: Forward pass
        log_probs = model(feature_vectors)

        # Step 3: Compute loss
        loss = loss_function(log_probs, labels)

        # Step 4: Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Store loss
        epoch_i_train_losses.append(loss.item())
test_feature_tensor = torch.tensor(all_test_feature_vectors, dtype=torch.float32)
true_test_labels_tensor = torch.tensor(test_labels, dtype=torch.int)

# Get predicted probabilities from the model
with torch.no_grad():  # No gradient calculation needed for evaluation
    test_probs = model(test_feature_tensor)

# Convert probabilities to binary labels
predicted_test_labels = model.logprob2label(test_probs)

# Convert tensors to lists for metric calculation
predicted_test_labels = predicted_test_labels.squeeze().tolist()
true_test_labels = true_test_labels_tensor.tolist()

# Calculate precision, recall, and F1-score
precision = precision_score(true_test_labels, predicted_test_labels)
recall = recall_score(true_test_labels, predicted_test_labels)
f1 = f1_score(true_test_labels, predicted_test_labels)

# Print results
print(f'Precision for Test: {precision:.4f}')
print(f'Recall for Test: {recall:.4f}')
print(f'F1-score for Test: {f1:.4f}')

100%|██████████| 4/4 [00:00<00:00, 1416.04it/s]


Precision for Test: 0.8462
Recall for Test: 0.8800
F1-score for Test: 0.8627


In [163]:
def featurize_text_5d(text):
    # Tokenize text
    words = text.lower().split()
    positive_count = sum(1 for word in words if word in positive_lexicon)
    negative_count = sum(1 for word in words if word in negative_lexicon)
    pronouns = {"i", "me", "my", "mine", "we", "us", "our", "ours", "you", "your", "yours"}
    pronoun_count = sum(1 for word in words if word in pronouns)
    contains_exclamation = 1.0 if "!" in text else 0.0
    log_word_count = math.log(len(words)) if len(words) > 0 else 0.0
    feature_vector = [positive_count, negative_count, pronoun_count, contains_exclamation, log_word_count]
    return feature_vector

In [164]:
all_test_feature_vectors_5d = [featurize_text(review) for review in test_texts]
all_test_feature_vectors_5d = normalize(all_test_feature_vectors_5d)

test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32).unsqueeze(1)  

for epoch in range(num_epochs):
    samples = list(zip(all_test_feature_vectors_5d, test_labels_tensor))
    random.shuffle(samples)
    batches = [samples[i:i + batch_size] for i in range(0, len(samples), batch_size)]

    epoch_i_train_losses = []

    for batch in tqdm(batches):
        feature_vectors, labels = zip(*batch)
        
        # Convert to tensors
        feature_vectors = torch.tensor(feature_vectors, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.float32).unsqueeze(1)  # Ensure labels are 2D
        
        # Step 1: Zero gradients
        model.zero_grad()

        # Step 2: Forward pass
        log_probs = model(feature_vectors)

        # Step 3: Compute loss
        loss = loss_function(log_probs, labels)

        # Step 4: Backpropagation and optimization
        loss.backward()
        optimizer.step()

        # Store loss
        epoch_i_train_losses.append(loss.item())
    
    # Print the average loss for this epoch
    print('Epoch:', epoch)
    print('Avg train loss:', sum(epoch_i_train_losses) / len(epoch_i_train_losses))

100%|██████████| 4/4 [00:00<00:00, 1169.63it/s]


Epoch: 0
Avg train loss: 0.35710281878709793


100%|██████████| 4/4 [00:00<00:00, 1631.71it/s]


Epoch: 1
Avg train loss: 0.37299320846796036


100%|██████████| 4/4 [00:00<00:00, 1295.64it/s]


...

Epoch: 97
Avg train loss: 0.35709116607904434


100%|██████████| 4/4 [00:00<00:00, 1373.83it/s]


Epoch: 98
Avg train loss: 0.4188472330570221


100%|██████████| 4/4 [00:00<00:00, 1505.76it/s]


Epoch: 99
Avg train loss: 0.3172076679766178


In [165]:
test_feature_tensor = torch.tensor(all_test_feature_vectors_5d, dtype=torch.float32)
true_test_labels_tensor = torch.tensor(test_labels, dtype=torch.int)

# Get predicted probabilities from the model
with torch.no_grad():  # No gradient calculation needed for evaluation
    test_probs = model(test_feature_tensor)

# Convert probabilities to binary labels
predicted_test_labels = model.logprob2label(test_probs)

# Convert tensors to lists for metric calculation
predicted_test_labels = predicted_test_labels.squeeze().tolist()
true_test_labels = true_test_labels_tensor.tolist()

# Calculate precision, recall, and F1-score
precision = precision_score(true_test_labels, predicted_test_labels)
recall = recall_score(true_test_labels, predicted_test_labels)
f1 = f1_score(true_test_labels, predicted_test_labels)

# Print results
print(f'Precision for Test: {precision:.4f}')
print(f'Recall for Test: {recall:.4f}')
print(f'F1-score for Test: {f1:.4f}')


Precision for Test: 0.8519
Recall for Test: 0.9200
F1-score for Test: 0.8846


# **REPORT**

## 1. Describe the details for your best performing model: parameters such as number of epochs, batch size, and learning rate. Compare and contrast with other parameter settings you used—feel free to use graphs or tables to compare. Why do you think it performed better than the other models?

Ans:
For my best performing model, I used the following hyperparameters:
I experimented with different configurations of these parameters to see their impact on performance. Below, I’ll describe the different settings I tried and compare the performance of each.

Comparison of Hyperparameter Settings
| Parameter Setting | Epochs | Batch Size | Learning Rate | F1 Score (Avg) | Train Loss (Avg) |
|-------------------|--------|------------|---------------|----------------|------------------|
| Model 1           | 100    | 16         | 0.1           | 0.86           | 0.44             |
| Model 2           | 100    | 20         | 0.01          | 0.86           | 0.37             |
| Model 3           | 100    | 16         | 0.005         | 0.90           | 0.43             |
| Model 4           | 200    | 16         | 0.05          | 0.87           | 0.26             |

When I increased the number of epochs with decreasing learning rate, I could see that I was getting a better F-1 score for Dev and Test. I could understand that when LR rate is slow there is a slow convergence of the model and I get a better result.
If LR is more, the slow convergence is not there and the values are dynamic and are oscillating throught the loss calculation process.



## 2. What do you think would happen if you didn’t normalize the feature vectors? Write down a guess for what you think would happen, and then run an experiment to test your intuitions and report back what you learned.

Ans:
If I don't normalize the feature vectors, I expect the model to perform poorly because features with larger values could dominate the learning process, making the model biased toward those features. The gradients might also become unbalanced, causing slow or unstable learning. It would have Magnitude differences, convergence issues etc.

After experimenting without normalizing the feature vectors, I could see that my hypothesis was correct. For the best model for which I was getting the F1 score above 90, it significantly dropped.





## 3. What would happen if you removed one of the features entirely and used a 5-dimensional feature vector? Choose one feature and remove it from your vector. Then, run another experiment and see what happens. Does the test F1 go up or down? Does the model converge slower, or faster? Report which feature you removed and what you learned.

Ans:
If I remove one of the features entirely and use a 5-dimensional feature vector, I expect the model to lose some information, which could lead to a decrease in performance. Removing a feature might make the model converge faster since there are fewer dimensions to learn, but it could also result in lower accuracy or a decrease in the F1 score because the model is missing important data.

I remove contains_no as a feature and I could see that the F1 score decreased a little bit. There was a significance difference when I removed positive or negative word count. The model convergence has a negligible speed increase for contains_no but model converged faster if I removed the positive and negative features.



## 4. Review Section 4.10, p. 18 of the textbook and then consider the resources we used for this task: for instance, the training data and the positive and negative lexicons. Did you notice any biases present in these resources? Can you think of any harms or unintended consequences (harmful or not) that this classifier could cause? There is no correct answer; just write a couple of sentences reflecting on this prompt.

Ans:
I noticed that there might be some biases in the positive and negative lexicons. For example, the negative lexicon appears to contain a broader variety of words, some of which may be mislabelled, leading to inaccurate predictions. This could result in unfair or skewed classifications, especially if certain negative words that are not actually related to reviews are included. One potential harm of this classifier is that it could incorrectly label neutral or complex statements as negative, influencing sentiment analysis in ways that could be misleading for users relying on the system for feedback or decision-making.
